https://docs.pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import os

In [ ]:
# Hyperparameters
batch_size = 128
z_dim = 100       # size of the noise vector
image_size = 28
channels = 1
epochs = 50
lr = 0.0002
betal = 0.5       # adam optimizer betal

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create output folder
os.makedirs('generated_images', exist_ok=True)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))   # Normalize between [-1, 1]
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.',train=True, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 11.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 344kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.20MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.69MB/s]


In [ ]:
class Generator(nn.Module):
  def __init__(self, z_dim):
    super(Generator, self).__init__()
    self.net = nn.Sequential(
        # Input: (N, z_dim, 1, 1)
        # Input: (input_dim (z), output_dim, kernel_size, stride, padding)
        nn.ConvTranspose2d(z_dim, 256, 7, 1, 0, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        # Output: (N, 256, 7, 7),

        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),

        nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
        nn.Tanh()  # Output range [-1, 1]
    )

  def forward(self, z):
    return self.net(z)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.net = nn.Sequential(
        # Input: (N, 1, 28, 28)
        nn.Conv2d(1, 64, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(64, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Flatten(),
        nn.Linear(128*7*7, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.net(x)

In [ ]:
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

# Binary Cross Entropy Loss
criterion = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(betal, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(betal, 0.999))

In [ ]:
def generate_and_save_images(epoch):
  generator.eval()
  with torch.no_grad():
    z = torch.randn(64, z_dim, 1, 1).to(device)
    fake_images = generator(z)
    fake_images = fake_images*0.5 + 0.5  # Denormalize to [0,1]
    save_image(fake_images, f'generated_images/sample_epoch_{epoch}.png',nrow=8)
  generator.train()

In [ ]:
k = 3 # Generator updates per iteration
p = 1 # Discriminator updates per iteration

In [ ]:
for epoch in range(1, epochs+1):
  for i, (real_imgs, _) in enumerate(train_loader):
    batch_size_curr = real_imgs.size(0)
    real_imgs = real_imgs.to(device)

    real = torch.ones(batch_size_curr, 1).to(device)
    fake = torch.zeros(batch_size_curr, 1).to(device)

    # Train Discriminator p times
    for _ in range(p):
      z = torch.randn(batch_size_curr, z_dim, 1, 1).to(device)
      fake_imgs = generator(z)

      # Real
      real_validity = discriminator(real_imgs)
      d_real_loss = criterion(real_validity, real)

      # Fake
      fake_validity = discriminator(fake_imgs.detach())
      d_fake_loss = criterion(fake_validity, fake)

      d_loss = d_real_loss + d_fake_loss

      optimizer_D.zero_grad()
      d_loss.backward()
      optimizer_D.step()

    # Train Generator k times
    for _ in range(k):
      z = torch.randn(batch_size_curr, z_dim, 1, 1).to(device)
      fake_imgs = generator(z)

      validity = discriminator(fake_imgs)
      g_loss = criterion(validity, real)

      optimizer_G.zero_grad()
      g_loss.backward()
      optimizer_G.step()

    if i % 200 == 0:
      print(f'[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]')

  # Save sample images
  generator.eval()
  with torch.no_grad():
      z = torch.randn(64, z_dim, 1, 1).to(device)
      fake_images = generator(z)
      fake_images = fake_images*0.5 + 0.5  # Denormalize to [0,1]
      save_image(fake_images, f'generated_images/sample_epoch_{epoch}.png',nrow=8)
  generator.train()

[Epoch 1/50] [Batch 0/469] [D loss: 1.1729] [G loss: 0.6255]
[Epoch 1/50] [Batch 200/469] [D loss: 1.1905] [G loss: 0.9280]
[Epoch 1/50] [Batch 400/469] [D loss: 1.0978] [G loss: 1.3831]
[Epoch 2/50] [Batch 0/469] [D loss: 1.2525] [G loss: 0.5058]
[Epoch 2/50] [Batch 200/469] [D loss: 1.1411] [G loss: 1.1209]
[Epoch 2/50] [Batch 400/469] [D loss: 1.1665] [G loss: 1.4688]
[Epoch 3/50] [Batch 0/469] [D loss: 1.1059] [G loss: 1.0693]
[Epoch 3/50] [Batch 200/469] [D loss: 1.1215] [G loss: 1.0138]
[Epoch 3/50] [Batch 400/469] [D loss: 1.3277] [G loss: 1.3711]
[Epoch 4/50] [Batch 0/469] [D loss: 1.1225] [G loss: 0.7394]
[Epoch 4/50] [Batch 200/469] [D loss: 1.0722] [G loss: 0.7851]
[Epoch 4/50] [Batch 400/469] [D loss: 1.0675] [G loss: 0.9508]
[Epoch 5/50] [Batch 0/469] [D loss: 1.0146] [G loss: 1.2689]
[Epoch 5/50] [Batch 200/469] [D loss: 1.0452] [G loss: 0.9317]
[Epoch 5/50] [Batch 400/469] [D loss: 1.0425] [G loss: 0.7659]
[Epoch 6/50] [Batch 0/469] [D loss: 1.1801] [G loss: 1.1874]
[Epo